# Basics of Python testing

The process of software development does not just consist of "write the code to solve a problem".
You want code that a) you are sure works correctly, b) is robust enough to deal with unexpected situations, c) is easy to extend or change,
d) is readable for others where "others" may mean you next year.

There is a broad range of important techniques that help you attain a reasonable level of confidence (and good conscience), including:

- well-defined problem (be sure that you know the scope of possible inputs, outputs and outcomes)
- separation of concerns (modular approach)
- readable code (meaningful names, avoid "fancy" constructs)
- explicit checks for presumptions that might be false
- good documentation (docstrings, tutorials, ...)
- (automated) testing

In this material, we are focusing on testing, answering the question **"Does my software really do what I expect it to do?"**

## Types of tests

The terminology is not fixed and some source mean different things under the same name, but roughly we
can distinguish the following:

- **unit testing** - on the lowest level, test that each component (function, class method) works as intended.
It is important that dependence on other components is mitigated - by careful design of functions (avoiding side effects
as much as possible), by dependency injection (allowing the components to swap the dependencies with dummy variants)
or by mocking (forcefully replacing dependencies with fake variants).

- **integration testing** - checking that a larger set of components works well together, usually in an environment
that resembles the production one (using real databases, files, ...)

- **functional testing** - mimick the whole run of an application, (theoretically) without any knowledge about
its code, provide some input / replay user interaction (whatever applicable) and check that the output you get is 
the one you want.

## What and when to test?

There is no simple answer (unless you take "More often than you think" for an answer) - it depends on the complexity of the system (more complexity -> more tests)
and also on the target audience (universal library with an unknown public audience requires thorough testing). There are multiple levels of rigorousness (and religiosity)
when it comes to testing:

- never: this is valid strategy for one-time scripts
- test the basic features as you use them to catch the biggest bugs (you can start with one or two integration/functional tests with your typical scenarios
 and when you break something, it will *probably* fail early)
- write tests that capture a certain bug and make sure that it won't appear again (while you are debugging,
  you may as well copy the code you execute and make it an automatic test)
- 100-% (or some other percentage) code coverage - you make sure that (almost) each line of your code gets executed at least once in the tests and you  will spot
a lot of bugs early. Reaching 100-% level does not assure your code is bug-free and focusing on this single metric can be dangerous.
- test-driven development - a nice (though rarely applied) concept of first writing the tests (that will fail initially) and then the code itself (make it pass).

Note that especially with the code used by others, it is important not just to test what happens with an expected and correct input (you probably code against this model)
but maybe even more important is to check what happens with unexpected input.

Also note that a properly written is a good form of documentation - a test can be an example how to use your functions / libraries.


## Assertions

This is a nice tool in the process of development - at certain point of the code you may make sure that you assumptions are correct by writing a simple yes/no test
called "assertion" (one line = one boolean condition).

When this line is executed, either the assumption holds (and nothing happens) or it fails (and an `AssertionError` is thrown).

In [ ]:
assert 14 > 10


The assumption is correct, so nothing (visible) happens.

But let's make another (admittedly stupid) assumption:

In [ ]:
assert 14 < 10

AssertionError: 

The code fails with an exception. The message is not much informative but at least we know that something failed and what code failed.

You may add an informative message about the purpose of the assertion if it is not obvious from the code itself.

In [ ]:
assert 14 < 10, "You lack the basic maths knowledge!"

AssertionError: You lack the basic maths knowledge!


**Warning:** Do not use assertions for testing input (or some application state) in production. It is very easy to run Python with assertions turned off. 
No checking occurs and a problem you think you are avoiding, silently passes on.

## Testing frameworks



In [ ]:
# Make sure these two get installed
%pip install pytest
%pip install hypothesis

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


### [unittest](https://docs.python.org/3/library/unittest.html)

This is a module included in the standard library of Python. We are not using it because even a simple test requires to implement a class and use its method to
check the results:

In [ ]:
%%file test_unittest.py

import math
import unittest

class TestSqrt(unittest.TestCase):   # Inherit from the base class
    def test_positive_number(self):
        y = math.sqrt(4)
        self.assertEqual(2, y)       # Use custom method

Overwriting test_unittest.py


We can run `unittest` from command line using the following command (actually running Python with the module as a main one).

In [ ]:
!python -m unittest test_unittest.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


### [pytest](https://docs.pytest.org/en/stable/contents.html)

This is an external library but also a de facto standard for testing. It allows to write test simply by making assertions (see above)... and much more.

In [ ]:
%%file test_pytest.py
import math

def test_positive_number():
    assert 2 == math.sqrt(4)

Overwriting test_pytest.py


We run the `pytest` executable in the command line (not in Python!) to get a summary of tests run and (if they happened) details about the failures.

In [ ]:
!pytest test_pytest.py

============================= test session starts ==============================
platform linux -- Python 3.7.9, pytest-6.2.2, py-1.10.0, pluggy-0.13.1
rootdir: /work/hilase-python-course-2021
plugins: hypothesis-6.1.1
collected 1 item                                                               

test_pytest.py .                                                         [100%]

============================== 1 passed in 0.03s ===============================


### [hypothesis](https://hypothesis.readthedocs.io)

This is an elegant library that works well with pytest and allows you to automatically test 
against a huge variety of inputs (this is very useful in scientific computing), including
edge cases.

For example, we can test out `sqrt` against a representative subset of positive integers:

In [ ]:
%%file test_hypothesis.py
import math

import numpy as np
from hypothesis import given
from hypothesis.strategies import integers
from hypothesis.extra.numpy import unsigned_integer_dtypes


@given(integers(0))
def test_positive_numbers(n: int):
    root = math.sqrt(n)
    assert np.isclose(root ** 2, float(n))

Overwriting test_hypothesis.py


We then normally run `pytest`:

In [ ]:
!pytest test_hypothesis.py

============================= test session starts ==============================
platform linux -- Python 3.7.9, pytest-6.2.2, py-1.10.0, pluggy-0.13.1
rootdir: /work/hilase-python-course-2021
plugins: hypothesis-6.1.1
collected 1 item                                                               

test_hypothesis.py .                                                     [100%]

============================== 1 passed in 0.34s ===============================


## Further reading

- [Getting Started With Testing in Python @ RealPython](https://realpython.com/python-testing/)
- [Testing by Honza Král] - recorded talk from Czech Python series of meetups "PyVo"
- [Property-Based Testing with `hypothesis`](https://bytes.yingw787.com/posts/2021/02/02/property_based_testing/)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=77a5caea-ff40-471d-8b4b-98dc66dd30c3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>